In [ ]:
import numpy as np
import os
import numba
import cupy as cp
#l=np.load('arr.npy')
#print(l)
# l=np.load('gradient.npy')
# print(len(l[0]))
from scipy.sparse import csr_matrix, save_npz, load_npz

In [ ]:
agg3=np.array(np.load("err3_in_topK.npy")) #, allow_pickle=True))
agg3=agg3.transpose()
A_conc3=np.load("A_conc3_in_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc3[i].todense())
A_conc3=np.array(A_con)
A_conc3= np.concatenate(A_conc3, axis=1)
#np.save("agg3.npy",agg3)

# del err1
# del err2
# del err3

In [ ]:
A=A_conc3[:, :1000]
# print(A.shape)
# print(A)
# print(A_conc1[1862324][0:1000])
A_conc3=A

In [ ]:
agg3_2=np.array(np.load("err3_in_2_topK.npy")) #, allow_pickle=True))
agg3_2=agg3_2.transpose()
A_conc3_2=np.load("A_conc3_in_2_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc3_2[i].todense())
A_conc3_2=np.array(A_con)
A_conc3_2= np.concatenate(A_conc3_2, axis=1)

In [ ]:
agg3_3=np.array(np.load("err3_in_3_topK.npy")) #, allow_pickle=True))
agg3_3=agg3_3.transpose()
A_conc3_3=np.load("A_conc3_in_3_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc3_3[i].todense())
A_conc3_3=np.array(A_con)
A_conc3_3= np.concatenate(A_conc3_3, axis=1)

In [ ]:
agg3_4=np.array(np.load("err3_in_4_topK.npy")) #, allow_pickle=True))
agg3_4=agg3_4.transpose()
A_conc3_4=np.load("A_conc3_in_4_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc3_4[i].todense())
A_conc3_4=np.array(A_con)
A_conc3_4= np.concatenate(A_conc3_4, axis=1)

In [ ]:
agg3 = np.concatenate((agg3, agg3_2), axis=1) #, agg3_3, agg3_4, agg3_3, agg3_4 # agg3_3, agg3_4
print(agg3.shape)
A_conc3 = np.concatenate((A_conc3, A_conc3_2), axis=1) # #, A_conc3_3, A_conc3_4, A_conc3_3, A_conc3_4
print(A_conc3.shape)

In [ ]:
del agg3_2
# del agg3_3
# del agg3_4
del A_conc3_2
# del A_conc3_3
# del A_conc3_4

In [ ]:
import torch
torch.cuda.empty_cache() 

In [ ]:
print(agg3.shape)

In [ ]:
#A_conc1= np.concatenate(A_conc1, axis=1)

In [ ]:
print(A_conc3.shape)

In [ ]:
print( np. __version__)

In [ ]:
import torch
if torch.cuda.is_available():
  dev = "cuda:3"
else:
  dev = "cpu"
device = torch.device(dev)

In [ ]:
from numba import cuda
from numba import jit
print(cuda.gpus)
numba.cuda.select_device( 3 )

In [ ]:
from numba import cuda
cuda.detect()
from numba import jit
#@cuda.jit
@jit #(target='cuda:1')
def matrix_comp(A,A_conc,d,iter_no):
    #for t in range(int(d/6)):
    for t in range(A_conc.shape[0]):
#     if (t%500000)==0:
#         print(t)
    #for j in range(user_no):
        A[t]=np.split(A_conc[t],iter_no)
        #A[t]=np.ma.row_stack (A[t])
    return A

In [ ]:
import time
d=11173962
iter_no=1000
user_no=5
A3=np.empty((int(d/6),iter_no,user_no))
start=time.time()
A3=matrix_comp(A3,A_conc3,d,iter_no)
end=time.time()
print(end-start)

In [ ]:
import torch
del A_conc3
torch.cuda.empty_cache()  

In [ ]:
from scipy import linalg
import time

#print(A)
#A=np.array(A)
#p,*_ = linalg.lstsq(A[:,:,0], e)
d=11173962

user_gradient=[]
check_gradient=[]
ind1=int(d/6)
num_users=user_no
start=time.time()   
for user in range(num_users):
#for user in user_no:
    user_gradient.append([])
    #check_gradient.append(original_gradient[user])
for t in range(ind1):
    #A_big[t]=np.array(A_big[t])
    #e[t]=np.array(e[t])
        #p,*_ = linalg.lstsq(A1[t], agg1[t])
    p, *_  = linalg.lstsq(A3[t], agg3[t][0:iter_no])
    #print(p)
    for user in range(num_users):
        user_gradient[user].append(p[user])
    
#check_gradient[user]=original_gradient[user]
end=time.time()
# ind1=int(d/3)
# estimated_grad=least_square(A1,agg1,ind1)
# end=time.time()

print(end-start)

In [ ]:
print(p)
print(len(user_gradient[0]))

In [ ]:
o=np.load("original_grad.npy")

In [ ]:
d=11173962
o=np.load("original_grad.npy")
#user_gradient=np.load("gradient33_0.1_randK.npy")
print(o[3][int(2*d/6):int(2*d/6+20)])
#print(user_gradient[2][int(d/6)-20:int(d/6)])
print(user_gradient[3][0:20])
print(np.linalg.norm(o[3][int(2*d/6):int(3*d/6)][:,0]-user_gradient[3]))

In [ ]:
np.save("gradient33.npy",user_gradient)
print("finish")

In [ ]:
d=11173962
u=np.load("gradient33_topK_1000.npy")
print(np.linalg.norm(o[3][int(2*d/6):int(3*d/6)][:,0]-u[3]))